In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("D:/DataSets ML/insurance/insurance.csv")

In [5]:
data['sex'].replace(to_replace=['male','female'],value =[0,1],inplace = True)

In [6]:
data['smoker'].replace(to_replace=['no','yes'],value =[0,1],inplace = True)

In [7]:
data['region'].replace(to_replace= data['region'].unique(),value = list(range(len(data['region'].unique()))),inplace = True)

In [8]:
#ohe - one hot encodeing
region_variable_ohe = np.eye(len(data['region'].unique()),len(data['region'].unique()))[data['region']]

In [9]:
processed_data = data.drop(['region'],axis = 1)

In [10]:
processed_data

,age,sex,bmi,children,smoker,charges
0,19,1,27.900,0,1,16884.92400
1,18,0,33.770,1,0,1725.55230
2,28,0,33.000,3,0,4449.46200
3,33,0,22.705,0,0,21984.47061
4,32,0,28.880,0,0,3866.85520
...,...,...,...,...,...,...
1333,50,0,30.970,3,0,10600.54830
1334,18,1,31.920,0,0,2205.98080
1335,18,1,36.850,0,0,1629.83350
1336,21,1,25.800,0,0,2007.94500


In [11]:
region_variable_df = pd.DataFrame(data = region_variable_ohe,columns = ['Region 0','Region 1','Region 2','Region 3'])

In [12]:
processed_data = pd.concat([processed_data,region_variable_df],axis =1)

In [13]:
gt_labels = processed_data['charges']

processed_data.drop(['charges'],axis = 1,inplace = True)

In [14]:
processed_data['charges'] = gt_labels

In [17]:
Y = np.array(processed_data['charges']).reshape(processed_data.shape[0],1)
X_transpose = np.array(processed_data.drop(['charges'],axis =1))

In [18]:
Y.shape

(1338, 1)

In [19]:
X_transpose = (X_transpose - np.mean(X_transpose,axis = 0))/np.std(X_transpose,axis = 0)

In [20]:
X_transpose_df = pd.DataFrame(data = X_transpose)

In [21]:
X_transpose_df
#now our data is coming dense after mean standar deciation , that what we want not sparsce data but dense data is good news for algo

,0,1,2,3,4,5,6,7,8
0,-1.438764,1.010519,-0.453320,-0.908614,1.970587,1.765481,-0.611324,-0.566418,-0.565267
1,-1.509965,-0.989591,0.509621,-0.078767,-0.507463,-0.566418,1.635795,-0.566418,-0.565267
2,-0.797954,-0.989591,0.383307,1.580926,-0.507463,-0.566418,1.635795,-0.566418,-0.565267
3,-0.441948,-0.989591,-1.305531,-0.908614,-0.507463,-0.566418,-0.611324,1.765481,-0.565267
4,-0.513149,-0.989591,-0.292556,-0.908614,-0.507463,-0.566418,-0.611324,1.765481,-0.565267
...,...,...,...,...,...,...,...,...,...
1333,0.768473,-0.989591,0.050297,1.580926,-0.507463,-0.566418,-0.611324,1.765481,-0.565267
1334,-1.509965,1.010519,0.206139,-0.908614,-0.507463,-0.566418,-0.611324,-0.566418,1.769076
1335,-1.509965,1.010519,1.014878,-0.908614,-0.507463,-0.566418,1.635795,-0.566418,-0.565267
1336,-1.296362,1.010519,-0.797813,-0.908614,-0.507463,1.765481,-0.611324,-0.566418,-0.565267


In [22]:
X_transpose_train = X_transpose[0: int(0.7*X_transpose.shape[0])]
X_transpose_cv = X_transpose[int(0.7*X_transpose.shape[0]): int(0.9*X_transpose.shape[0])]
X_transpose_test = X_transpose[int(0.9*X_transpose.shape[0]):]

In [23]:
#Y_train = Y[0: int(0.7*X_transpose.shape[0])]
#now we are normalising Y_train
Y_train =  (Y[0: int(0.7*X_transpose.shape[0])] -np.mean( Y[0: int(0.7*X_transpose.shape[0])]))/np.std( Y[0: int(0.7*X_transpose.shape[0])])
Y_cv= Y[int(0.7*X_transpose.shape[0]) :int(0.9*X_transpose.shape[0])]
Y_test = Y[int(0.9*X_transpose.shape[0]):]

In [24]:
N_train = X_transpose_train.shape[0]
m = X_transpose_train.shape[1]

In [25]:
def predicted_answers(theta0, theta,X_transpose):
    return theta0 + np.matmul(X_transpose,theta)

In [45]:
def neg_log_loss(labels,predicted_labels, theta,reg_strength):
    E = labels -  predicted_labels
    return (1/labels.shape[0])*np.matmul(E.T,E) + (reg_strength*(np.linalg.norm(theta))**2/2)

In [32]:
Y_train.shape

(936, 1)

In [41]:
def fit(step_size,reg_strength):
    theta0_initial = 0
    theta_initial = np.zeros((m,1))
    epsilon = step_size
    tol = 10**(-6)
    iteration = list()
    neg_log_loss_history = list()
    iteration_number = 0
    mini_batch_size = 36
    time_steps = N_train//mini_batch_size
    epoch_counter = 0
    invalid_value_flag = 0
    
    while(True):
        for i in range(0,time_steps):
            rand_indices = np.random.choice(a=np.arange(0,N_train),size = mini_batch_size,replace=False)
            X_transpose_mini_batch = X_transpose[rand_indices]
            train_labels_mini_batch = Y_train[rand_indices]
            
            Predicted_answers_initial = predicted_answers(theta0_initial, theta_initial,X_transpose_mini_batch)
            theta0_final = theta0_initial - epsilon*np.mean(Predicted_answers_initial-train_labels_mini_batch)
            theta_final = theta_initial - epsilon*((1/mini_batch_size)*np.matmul(X_transpose_mini_batch.T,(Predicted_answers_initial-train_labels_mini_batch))
                                                   + (reg_strength*theta_initial))
            
            neg_log_loss_initial = neg_log_loss(train_labels_mini_batch, Predicted_answers_initial, theta_initial, reg_strength)
            
            Predicted_answers_final = predicted_answers(theta0_final,theta_final,X_transpose_mini_batch)
            neg_log_loss_final = neg_log_loss(train_labels_mini_batch, Predicted_answers_final, theta_final, reg_strength)
            
            if np.isnan(neg_log_loss_final[0][0]) == True:
                invalid_value_flag =1
                break
            
            theta0_initial = theta0_final
            theta_initial = theta_final
            
            iteration.append(iteration_number)
            neg_log_loss_history.append(neg_log_loss_initial[0][0])
            
            iteration_number = iteration_number + 1
            
            #print(" Iteratins:", iteration_number, "Mean Squared Error : ", neg_log_loss[0][0])
        
        epoch_counter = epoch_counter +1
        
        if abs(neg_log_loss_initial - neg_log_loss_final) <tol or invalid_value_flag ==1:
            print(" \n\n Epochs -", epoch_counter, "Mean Squared Error Loss - ", neg_log_loss_initial[0][0],"\n\n")
            break
        
        
        return {(step_size,reg_strength):[theta0_final, theta_final]}
            
            
        

In [65]:
param_config = fit(0.00001,0.01)

In [55]:
def compute_mse(labels, predicted_labels):
    E = (labels - predicted_labels)
    return (1/labels.shape[0])*np.matmul(E.T,E)

In [61]:
def rmse(labels,trained_theta0,trained_theta,X_transpose):
    predicted_labels = predicted_answers(trained_theta0, trained_theta, X_transpose)
    return np.sqrt(compute_mse(labels,predicted_labels))

In [46]:
grid_search_results = list()
for step_size in [0.000001, 0.00001]:
    for reg_hyp_param in [0.0001,0.001,0.01,0.1,1,10,100]:
        grid_search_results.append(fit(step_size,reg_hyp_param))

 

 Epochs - 1 Mean Squared Error Loss -  0.4289934572698576 




In [67]:
params = list(param_config.values())

In [101]:
#params[0][1]
Y_train.shape

(936, 1)

In [102]:
train_rmse = rmse(Y_train,params[0][0],params[0][1],X_transpose_train)

In [103]:
train_rmse

array([[0.99825274]])

In [68]:
Y_cv = (Y_cv - np.mean(Y_cv))/np.std(Y_cv)

In [69]:
cv_rmse = rmse(Y_cv,params[0][0],params[0][1],X_transpose_cv)

In [70]:
cv_rmse

array([[0.99983586]])

In [71]:
grid_search_cv_results = dict()
for configs in grid_search_results:
    if configs is not None:
        trained_params = list(configs.values())
        grid_search_cv_results[tuple(configs.keys())] = rmse(Y_cv, trained_params[0][0],
                                                            trained_params[0][1], X_transpose_cv
                                                            )

In [72]:
grid_search_cv_results

{((1e-06, 0.001),): array([[0.99998108]]),
 ((1e-06, 0.01),): array([[0.99998082]]),
 ((1e-06, 0.1),): array([[0.99998119]]),
 ((1e-06, 1),): array([[0.99998087]]),
 ((1e-06, 10),): array([[0.99998436]]),
 ((1e-06, 100),): array([[0.99998025]]),
 ((1e-05, 0.0001),): array([[0.99979583]]),
 ((1e-05, 0.001),): array([[0.99980531]]),
 ((1e-05, 0.01),): array([[0.99982154]]),
 ((1e-05, 0.1),): array([[0.99980326]]),
 ((1e-05, 1),): array([[0.99982369]]),
 ((1e-05, 10),): array([[0.99982377]]),
 ((1e-05, 100),): array([[0.99981807]])}

In [73]:
# RANDOM SEARCH
random_search_results = list()
for i in range(10):
    step_size = np.random.uniform(low = 10**(-7), high = 10**(-4))
    reg_hyp_param = np.random.uniform(low = 10**(-4), high = 10**(3))
    random_search_results.append(fit(step_size,reg_hyp_param))

In [ ]:
random_searc

In [81]:
random_search_cv_results = dict()
for configs in random_search_results:
    if configs is not None:
        trained_params = list(configs.values())
        random_search_cv_results[tuple(configs.keys())] = rmse(Y_cv, trained_params[0][0],
                                                            trained_params[0][1], X_transpose_cv
                                                            )

In [82]:
random_search_cv_results

{((4.0582129779031345e-05, 911.8191346764794),): array([[0.99951474]]),
 ((5.0871039311073556e-05, 798.755666139493),): array([[0.99934135]]),
 ((2.6429770721657272e-05, 166.54024779021836),): array([[0.99947882]]),
 ((9.209337728971808e-05, 144.25955246224555),): array([[0.99857869]]),
 ((6.51750860697048e-05, 686.0665168197987),): array([[0.99933749]]),
 ((2.9164581318776655e-05, 837.8099655993876),): array([[0.9995844]]),
 ((5.1940662712210684e-05, 345.9107969527868),): array([[0.99926668]]),
 ((3.208769649805189e-05, 475.9504650940806),): array([[0.99950821]]),
 ((8.791277311952274e-05, 74.31017463478695),): array([[0.99851354]]),
 ((8.370353278496505e-05, 644.3531124640608),): array([[0.99909686]])}

In [94]:
from sklearn.preprocessing import PolynomialFeatures

In [95]:
feature_engineering_obj = PolynomialFeatures()

In [96]:
X_transpose_quadratic = feature_engineering_obj.fit_transform(X= X_transpose)

In [97]:
X_transpose_quadratic = X_transpose_quadratic[:,1:]

In [89]:
X_transpose_quadratic_df = pd.DataFrame(data = X_transpose_quadratic)

In [117]:
X_transpose_quadratic_train = X_transpose_quadratic[0: int(0.7*X_transpose_quadratic.shape[0])]
X_transpose_quadratic_cv = X_transpose_quadratic[int(0.7*X_transpose_quadratic.shape[0]):int(0.9*X_transpose_quadratic.shape[0])]


In [87]:
#feature_engineering_obj = PolynomialFeatures(degree = 3)

In [88]:
from sklearn.linear_model import LinearRegression

In [100]:
lr_obj = LinearRegression()

In [102]:
lr_obj.fit(X=X_transpose_quadratic_train,y =Y_train)

LinearRegression()

In [109]:
theta_trained = lr_obj.coef_

In [111]:
theta_trained = theta_trained.reshape(theta_trained.shape[1],1)

In [106]:
theta0_trained = lr_obj.intercept_

In [115]:
train_rmse = rmse(Y_train, theta0_trained, theta_trained, X_transpose_quadratic_train)

In [116]:
train_rmse

array([[0.37164597]])

In [118]:
cv_rmse = rmse(Y_cv, theta0_trained,theta_trained, X_transpose_quadratic_cv)

In [119]:
cv_rmse

array([[0.44760209]])